In [1]:
import time
import requests
import xml.etree.ElementTree as ET
import pymysql
import schedule


# API 인증키
API_KEY = "Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw%3D%3D"
# API 공통파라미터
API_COMMON_PARAMS = {"serviceKey" : requests.utils.unquote(API_KEY), "pageNo" : "1"} # service key가 UTF-8형식이므로 decode하여 넘긴다.(get() 내부에서 encoding됨)

CURRENT_DATE = time.strftime("%Y%m%d") # yyyymmdd
BASE_CITY_CODE = "CJU" # 제주

# API 요청
def get_response_xml(url, params):
    response = requests.get(url, params=params)
    print(response.url)
    print(params)
    
    if response.status_code == 200:
        return ET.fromstring(response.text)
    else:
        return None
    
# xml에서 필요한 데이터만 dict형 배열에 저장
def get_data(result_xml, col_names):
    result_list = []
    
    for item in result_xml.findall("./body/items/item"):
        result = {}
        
        for xml_col_name, table_col_name in col_names.items():
            try:
                if table_col_name.endswith("_date"):
                    result[table_col_name] = item.find(xml_col_name).text[:10]
                else:
                    result[table_col_name] = item.find(xml_col_name).text
            except:
                continue
            
        result_list.append(result)
            
    return result_list

# 국내선 운항 스케쥴(제주 출발) 컬럼
def get_departure_schedule_columns():
    return {"airlineKorean" : "airline_name",
            "arrivalcity" : "arrival_city",
            "domesticNum" : "flight_num",
            "domesticArrivalTime" : "arrival_time",
            "domesticStartTime" : "departure_time",
            "domesticStdate" : "start_date",
            "domesticEddate" : "end_date",
            "domesticMon" : "mon_yn",
            "domesticTue" : "tue_yn",
            "domesticWed" : "wed_yn",
            "domesticThu" : "thu_yn",
            "domesticFri" : "fri_yn",
            "domesticSat" : "sat_yn",
            "domesticSun" : "sun_yn"}

# 국내선 운항 스케쥴(제주 도착) 컬럼
def get_arrival_schedule_columns():
    return {"airlineKorean" : "airline_name",
            "startcity" : "departure_city",
            "domesticNum" : "flight_num",
            "domesticArrivalTime" : "arrival_time",
            "domesticStartTime" : "departure_time",
            "domesticStdate" : "start_date",
            "domesticEddate" : "end_date",
            "domesticMon" : "mon_yn",
            "domesticTue" : "tue_yn",
            "domesticWed" : "wed_yn",
            "domesticThu" : "thu_yn",
            "domesticFri" : "fri_yn",
            "domesticSat" : "sat_yn",
            "domesticSun" : "sun_yn"}

# 실시간 운항정보 컬럼
def get_flight_status_columns():
    return {"airFln" : "flight_num",
            "airlineKorean" : "airline_name",
            "airport" : "airport_code",
            "city" : "city_code",
            "arrivedKor" : "arrived_airport_name",
            "boardingKor" : "boarding_airport_name",
            "std" : "scheduled_time",
            "io" : "inout_code",
            "rmkEng" : "flight_status_eng",
            "rmkKor" : "flight_status_kor"}

# 국내선 운항 스케쥴(제주 출발) 조회(/FlightScheduleList/getDflightScheduleList)
def get_departure_schedule_list():
    # 가져올 데이터의 항목명 정의
    col_name = get_departure_schedule_columns()
    params = API_COMMON_PARAMS.copy()
    params["schDate"] = CURRENT_DATE
    params["schDeptCityCode"] = BASE_CITY_CODE
    params["numOfRows"] = 999 # 한번에 출력할 최대 데이터수(*이 값 이상은 안넘어갈거 같아서 우선 대충 설정..)
    url = "http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList"
    
    result_xml = get_response_xml(url, params)
    
    return get_data(result_xml, col_name)

# 국내선 운항 스케쥴(제주 도착) 조회(/FlightScheduleList/getDflightScheduleList)
def get_arrival_schedule_list():
    col_name = get_arrival_schedule_columns()
    params = API_COMMON_PARAMS.copy()
    params["schDate"] = CURRENT_DATE
    params["schArrvCityCode"] = BASE_CITY_CODE
    params["numOfRows"] = 999 # 한번에 출력할 최대 데이터수(*이 값 이상은 안넘어갈거 같아서 우선 대충 설정..)
    url = "http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList"
    
    result_xml = get_response_xml(url, params)
    
    return get_data(result_xml, col_name)

# 실시간 운항정보 조회(/FlightStatusList/getFlightStatusList)
def get_flight_status_list():
    # 가져올 데이터의 항목명
    col_name = get_flight_status_columns()
    params = API_COMMON_PARAMS.copy()
    params["schLineType"] = "D" # D:국내
    params["schAirCode"] = BASE_CITY_CODE
    params["numOfRows"] = 999 # 한번에 출력할 최대 데이터수(*이 값 이상은 안넘어갈거 같아서 우선 대충 설정..)
    url = "http://openapi.airport.co.kr/service/rest/FlightStatusList/getFlightStatusList"
    
    result_xml = get_response_xml(url, params)
    
    return get_data(result_xml, col_name)



In [2]:
# departure_list = get_departure_schedule_list()
# arrival_list = get_arrival_schedule_list()
# status_list = get_flight_status_list()

In [3]:
# db 연결
def connect_db():
    return pymysql.connect(host='127.0.0.1',
                           user='root',
                           port=3307,
                           password='0313', # database 접속 비밀번호
                           db='flightdb',
                           charset='utf8mb4',
                           use_unicode=True,
                           cursorclass=pymysql.cursors.DictCursor)

# insert 쿼리 작성 및 실행
def insert_data(table_name, data_list):
    conn = connect_db()
    cur = conn.cursor()
    
    sql = "INSERT INTO {}({}) VALUES ({})".format(table_name, ", ".join(data_list[0].keys()), ", ".join(["%s"] * len(data_list[0])))
    data_list = [tuple(data_list[i].values()) for i in range(len(data_list))] # list[dict]형을 list[tuple]형으로 변환(쿼리실행용)
    print(sql)
    print(data_list[0])
    
    cur.executemany(sql, data_list)
    conn.commit()

    print(table_name, " 테이블 입력건수 >>", cur.rowcount)

    cur.close()
    conn.close()

# 프로그램 실행
def main():
    departure_list = get_departure_schedule_list()
    arrival_list = get_arrival_schedule_list()
    status_list = get_flight_status_list()
    
    insert_data("flight_schedule", departure_list)
    insert_data("flight_schedule", arrival_list)
    insert_data("flight_status", status_list)
    
schedule.every().day.at("12:29").do(main) # 매일 오후10시 실행되도록 job 등록
#schedule.every(1).minutes.do(main) # *테스트용 -> 1분에 한 번 job실행

while True:
    schedule.run_pending()
    time.sleep(1)

http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList?serviceKey=Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw%3D%3D&pageNo=1&schDate=20200413&schDeptCityCode=CJU&numOfRows=999
{'serviceKey': 'Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw==', 'pageNo': '1', 'schDate': '20200413', 'schDeptCityCode': 'CJU', 'numOfRows': 999}
http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList?serviceKey=Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw%3D%3D&pageNo=1&schDate=20200413&schArrvCityCode=CJU&numOfRows=999
{'serviceKey': 'Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6BSht44RZMRnxVrw4NdAA07lo6kd9fNT6ThCPql8Ia0a2EBDlbw==', 'pageNo': '1', 'schDate': '20200413', 'schArrvCityCode': 'CJU', 'numOfRows': 999}
http://openapi.airport.co.kr/service/rest/FlightStatusList/getFlightStatusList?serviceKey=Hl9nCtSJsJYpZKSXI1I4S3Ucr7gOPCG6dCX6

TypeError: not enough arguments for format string